In [1]:
import mlflow

In [2]:
mlflow.set_tracking_uri('http://ec2-54-212-194-129.us-west-2.compute.amazonaws.com:5000') 

### Tracking del Modelo

In [51]:
client = mlflow.tracking.MlflowClient()

In [52]:
experiments = (client.search_experiments())

In [53]:
experiments = list(filter(lambda k: k.name == 'experiment_et_tunned', experiments))[0]

In [54]:
experiments

<Experiment: artifact_location='s3://mlflow-artifacts-remote-buckets/3', creation_time=1688232918326, experiment_id='3', last_update_time=1688232918326, lifecycle_stage='active', name='experiment_et_tunned', tags={}>

In [55]:
from mlflow.entities import ViewType
runs = mlflow.MlflowClient().search_runs(
    experiment_ids=experiments.experiment_id,
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results = 5,
    order_by=["metrics.MAE ASC"]
)

In [56]:
run = runs[0]

In [57]:
run

<Run: data=<RunData: metrics={'MAE': 13590.2301,
 'MAPE': 0.4664,
 'MSE': 4905591187.8898,
 'R2': 0.586,
 'RMSE': 51017.6582,
 'RMSLE': 0.4775,
 'TT': 0.3}, params={'bootstrap': 'False',
 'ccp_alpha': '0.0',
 'criterion': 'squared_error',
 'max_depth': '10',
 'max_features': 'sqrt',
 'max_leaf_nodes': 'None',
 'max_samples': 'None',
 'min_impurity_decrease': '0.2',
 'min_samples_leaf': '5',
 'min_samples_split': '9',
 'min_weight_fraction_leaf': '0.0',
 'n_estimators': '170',
 'n_jobs': '-1',
 'oob_score': 'False',
 'random_state': '123',
 'verbose': '0',
 'warm_start': 'False'}, tags={'Run ID': '1c6905460330486c8381772ede0f0a52',
 'Run Time': '121.36',
 'Source': 'tune_model',
 'URI': '29aba7c7',
 'USI': '083d',
 'mlflow.log-model.history': '[{"run_id": "1c6905460330486c8381772ede0f0a52", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2023-07-01 17:38:09.422448", "flavors": '
                             '{"python_functio

In [58]:
run.data.tags['Run ID']

'1c6905460330486c8381772ede0f0a52'

In [59]:
logg_model = 'runs:/' + run.data.tags['Run ID'] + '/model' 
load_model = mlflow.pyfunc.load_model(logg_model)

2023/07/01 12:45:00 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.4.1, required: mlflow==2.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [60]:
import pandas as pd
data = pd.read_csv('raw.csv')

In [61]:
load_model.predict(data.drop(columns=['Rent']))

array([12541.20308517, 10993.68754114, 11268.26956021, ...,
       44535.77033338, 44219.855083  , 16135.16424371])

### Registro de Modelo

In [62]:
logg_model

'runs:/1c6905460330486c8381772ede0f0a52/model'

In [63]:
register_name = 'extra_trees'

In [64]:
mlflow.register_model(logg_model, register_name)

Registered model 'extra_trees' already exists. Creating a new version of this model...
2023/07/01 12:45:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: extra_trees, version 2
Created version '2' of model 'extra_trees'.


<ModelVersion: aliases=[], creation_timestamp=1688233538061, current_stage='None', description='', last_updated_timestamp=1688233538061, name='extra_trees', run_id='1c6905460330486c8381772ede0f0a52', run_link='', source='s3://mlflow-artifacts-remote-buckets/3/1c6905460330486c8381772ede0f0a52/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [68]:
version = 2

In [69]:
logg_model_uri = 'models:/' + register_name + f'/{version}' 
model = mlflow.pyfunc.load_model(logg_model_uri)

2023/07/01 12:46:10 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.4.1, required: mlflow==2.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [70]:
model.predict(data.drop(columns=['Rent']))

array([12541.20308517, 10993.68754114, 11268.26956021, ...,
       44535.77033338, 44219.855083  , 16135.16424371])

In [71]:
client = mlflow.MlflowClient()
client.update_model_version(
        name='extra_trees',
        version=2,
        description='This model uses extra trees algorithm'
)

<ModelVersion: aliases=[], creation_timestamp=1688233538061, current_stage='None', description='This model uses extra trees algorithm', last_updated_timestamp=1688233693242, name='extra_trees', run_id='1c6905460330486c8381772ede0f0a52', run_link='', source='s3://mlflow-artifacts-remote-buckets/3/1c6905460330486c8381772ede0f0a52/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [72]:
client = mlflow.MlflowClient()
client.transition_model_version_stage(
    name='extra_trees', version=2, stage='Production'
)

<ModelVersion: aliases=[], creation_timestamp=1688233538061, current_stage='Production', description='This model uses extra trees algorithm', last_updated_timestamp=1688233785976, name='extra_trees', run_id='1c6905460330486c8381772ede0f0a52', run_link='', source='s3://mlflow-artifacts-remote-buckets/3/1c6905460330486c8381772ede0f0a52/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [74]:
client = mlflow.MlflowClient()
client.transition_model_version_stage(
    name='extra_trees', version=1, stage='Staging'
)

<ModelVersion: aliases=[], creation_timestamp=1688233261413, current_stage='Staging', description='', last_updated_timestamp=1688233854681, name='extra_trees', run_id='be43d985bdac4e85a6b2b3e44689803b', run_link='', source='s3://mlflow-artifacts-remote-buckets/2/be43d985bdac4e85a6b2b3e44689803b/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [75]:
model_name = 'extra_trees'
stage = 'Production'

model = mlflow.pyfunc.load_model(model_uri=f'models:/{model_name}/{stage}')
model.predict(data.drop(columns=['Rent']))

2023/07/01 12:53:26 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.4.1, required: mlflow==2.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


array([12541.20308517, 10993.68754114, 11268.26956021, ...,
       44535.77033338, 44219.855083  , 16135.16424371])

In [76]:
data

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner
...,...,...,...,...,...,...,...,...,...,...,...,...
4741,2022-05-18,2,15000,1000,3 out of 5,Carpet Area,Bandam Kommu,Hyderabad,Semi-Furnished,Bachelors/Family,2,Contact Owner
4742,2022-05-15,3,29000,2000,1 out of 4,Super Area,"Manikonda, Hyderabad",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Owner
4743,2022-07-10,3,35000,1750,3 out of 5,Carpet Area,"Himayath Nagar, NH 7",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Agent
4744,2022-07-06,3,45000,1500,23 out of 34,Carpet Area,Gachibowli,Hyderabad,Semi-Furnished,Family,2,Contact Agent
